Krok 0: zebranie danych wejściowych

Zrodlo danych: Sentinel Pobrane z wykorzystaniem: Creodias Level-1C Liczba warstw 13 Kroki wykonane przed analizą: Zmiana formatu warstw z jp2 na png ze względu na bezproblemową współpracę z OpenCV Skompresowanie wszystkich warstw do rozdzielczości 1830x1830 Umieszczenie wszystkich warstw w katalogu Data

Krok 1: Klasyfikacja terenu: Wykorzystano QGIS oraz wtyczkę QuickOSM Z Open Street Map pobrano następujące warstwy: natural water landuse forest landuse farmland Ograniczono ich zasięg do zasięgu zdjęcia z Sentinel

TODO: Weryfikacja zgodności danych OSM i Sentinel i ewentualne naniesienie poprawek. Wstępne oględziny nie wykazały znacznych odstępst.

Krok 3. Wczytanie danych do Pythona

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import preprocesing as pre
from configparser import ConfigParser
from helpers import plot_MinMaxAvg
from helpers import plot_values_histogram
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from helpers import undersampling
from helpers import show_target_pred_dif
from configparser import ConfigParser
import matplotlib.pyplot as plt

In [ ]:
from helpers import show_classes_distribution

In [ ]:
config = ConfigParser()
config.read('config.ini')
input_dir = config['main']['input_dir']
class_file = config['main']['classification_data']
dx = int(config['main']['x_size'])
dy = int(config['main']['y_size'])
x_start = int(config['main']['x_start'])
y_start = int(config['main']['y_start'])
csv_data_file = config['main']['csv_data_file']

In [ ]:
data, columns_names = pre.images_to_numpy(input_dir, dx, dy, x_start, y_start)

In [ ]:
plot_MinMaxAvg(data, columns_names)

In [ ]:
plot_values_histogram(data, columns_names, ncols=5)

In [ ]:
 data = pre.remove_outstandings(data)

In [ ]:
plot_MinMaxAvg(data, columns_names)

In [ ]:
plot_values_histogram(data, columns_names, ncols=5)

In [ ]:
classes, class_names = pre.get_classes(class_file, dx, dy, x_start, y_start)
other = (1 - classes.any(axis=1).astype(int)).reshape(-1,1)
class_names += ['other']
pre.add_classes_to_config(config, class_names)
columns_names += class_names 

In [ ]:
show_classes_distribution(np.concatenate((classes, other), axis=1),class_names)

In [ ]:
data = np.concatenate((data, classes, other), axis=1)
data = pd.DataFrame(data, columns=columns_names)
data.to_csv(csv_data_file)

Dane zostały przygotowane teraz możemy spróbować je sklasyfikować

# Klasyfikacja obszaru z wykorzystaniem lasów losowych

In [ ]:
config = ConfigParser()
config.read('config.ini')
x_size = int(config['main']['x_size'])
y_size = int(config['main']['y_size'])
clases_names = list(config['classes'].values())
nr_of_classes = len(clases_names)
csv_data_file = config['main']['csv_data_file']

In [ ]:
df_unchange = pd.read_csv(csv_data_file)
df = undersampling(df_unchange,clases_names)
df = df_unchange
X = df.iloc[:,:-nr_of_classes].to_numpy()
Y = df.iloc[:,-nr_of_classes:].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(verbose=1,max_depth=5, random_state=0)
clf.fit(X_train, y_train)

In [ ]:
y_pred_RF = clf.predict(X_test)

# Klasyfikacja obszaru z wykorzystaniem sieci neuronowych

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from helpers import undersampling
from helpers import show_target_pred_dif
from configparser import ConfigParser

In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=1000)

In [ ]:
y_pred_DL = model.predict(X_test)

# Klasyfikacja obszaru z wykorzystaniem samoorganizujących się map

In [ ]:
from minisom import MiniSom
x_som, y_som = 4,4
som = MiniSom(x=x_som, y=y_som, input_len=X.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(X)
som.train_random(X, num_iteration=100000, verbose=False)

In [ ]:
fig, ax = plt.subplots()
ax.set_title('SOM')
plt.imshow(som.distance_map())
for (i, j), z in np.ndenumerate(som.distance_map()):
    if z > 0.10:
        ax.text(j, i, '{:0.2f}'.format(som.distance_map()[i,j]), ha='center', va='center',color = 'white')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_pred_SOM = [som.winner(x) for x in X]
y_pred_SOM = np.array([i[0]*100 + i[1] for i in y_pred_SOM ])
y_pred_SOM = enc.fit_transform(y_pred_SOM.reshape(-1, 1)).toarray()

In [ ]:
cm = confusion_matrix(Y.argmax(axis=1), y_pred_SOM.argmax(axis=1))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, axs = plt.subplots(figsize=(8,6), ncols = 2, nrows=2)
plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.2, wspace=0.4)
vmax = np.max(cm)/(0.25*len(y_pred_SOM))
for i, ax in enumerate(axs.flatten()):
    data = cm[i,:].reshape((x_som,y_som))/(0.25*len(y_pred_SOM))
    subplot = ax.matshow(data,vmin=0, vmax=vmax, cmap='BuGn')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    ax.set_title(df.columns[-4+i])
    for (i, j), z in np.ndenumerate(data):
        if(z>0.01):
            ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center',color = 'white')
    fig.colorbar(subplot, cax=cax)

In [ ]:
from helpers import IoU
from helpers import metrics_matrix

clusstered = np.zeros((x_som,y_som,3))
metrix_IoU = metrics_matrix(Y, y_pred_SOM, IoU)
clusstered[...,2]=metrix_IoU[:,0].reshape((x_som,y_som))
clusstered[...,1]=metrix_IoU[:,1].reshape((x_som,y_som))
clusstered[...,0]=metrix_IoU[:,2].reshape((x_som,y_som))
fig, ax = plt.subplots()
ax.set_title('Intersection over union')
plt.imshow(clusstered)
for (i, j, k), z in np.ndenumerate(clusstered):
    if z > 0.10:
        ax.text(j, i, '{:0.2f}'.format(max(clusstered[i,j,:])), ha='center', va='center',color = 'white')

In [ ]:
best_3 = [metrix_IoU[:,i].argsort()[-3:][::-1] for i in range(3)]
enc = OneHotEncoder()
y_pred_SOM = [som.winner(x) for x in X]
y_pred_SOM = np.array([i[0]*100 + i[1] for i in y_pred_SOM ])
y_pred_SOM = enc.fit_transform(y_pred_SOM.reshape(-1, 1)).toarray()

In [ ]:
Y_present = y_pred_SOM.reshape((x_size,y_size,y_pred_SOM.shape[1]))

In [ ]:
best_1 = [best_3[i][0] for i in range(3)]

In [ ]:
y_target = Y[...,:-1].reshape((x_size,y_size,3))
y_pred = Y_present[...,best_1]

In [ ]:
show_target_pred_dif(y_target,y_pred_SOM)